In [21]:
import openai
import os
import sys
import termcolor
sys.path.append("../")
from magpie.prompt_planner.prompts import mp_prompt_thinker_coder_muk as mptc
from magpie.prompt_planner import conversation
from magpie.prompt_planner import confirmation_safe_executor

API_KEY = os.environ['CORRELL_API_KEY']

In [6]:
import numpy as np
from PIL import Image
import base64
import io
import cv2 

def encode_image(img):
    # Convert NumPy array to base64 encoded string
    return base64.b64encode(img).decode('utf-8')


i = encode_image(np.array(Image.open("static/favicon.jpg")))

In [7]:
MODEL = "gpt-3.5-turbo"
import dataclasses
from typing import Any

In [8]:
@dataclasses.dataclass(frozen=True)
class TaskConfig:
  client: None
  prompts: dict[str, type[Any]]

ALL_TASKS = {
    'magpie': TaskConfig(
        client=None,
        prompts={
            'thinker_coder': mptc.PromptThinkerCoder,
        },
    ),
}


In [17]:
safe_executor = confirmation_safe_executor.ConfirmationSafeExecutor()
task_config = ALL_TASKS['magpie']
openai.api_key = API_KEY
prompt = task_config.prompts['thinker_coder']
MODEL = "gpt-3.5-turbo"

In [24]:
try:
    prompt_model = prompt(
        None, executor=safe_executor
    )
    conv = conversation.Conversation(prompt_model, MODEL)
    while True:
      user_command = input(termcolor.colored("User: ", "red", attrs=["bold"]))
      try:
        response = conv.send_command(user_command)
        print(termcolor.colored("Magpie: ", "green", attrs=["bold"]) + response + "\n")
      except Exception as e:  # pylint: disable=broad-exception-caught
        print("Planning failed, try something else... " + str(e) + "\n")
        continue
      
      try:
        prompt_model.code_executor(response)
      except Exception as e:  # pylint: disable=broad-exception-caught
        print("Execution failed, try something else... " + str(e) + "\n")

except Exception as e:
    print(e)
    prompt_model = None

LLM0 queried
[start of description]
* This is a new grasp.
* This grasp should be for picking up a grape.
* This is a complete grasp.
* This grasp does not contain multiple grasps.
* This grasp is for an object with low compliance.
* The object has an approximate mass of 5 grams.
* The object has an approximate spring constant of 50 Newtons per meter.
* The gripper and object have an approximate friction coefficient of 0.2.
* This grasp should set the goal aperture to 5 mm.
* Based on object mass and friction coefficient, grasp should initially set the contact force to 1 Newtons.
* If the gripper slips, this grasp should close an additional 2 mm.
* If the gripper slips, this grasp should increase the output force by 0.1 Newtons.
[end of description]

LLM1 queried
```python
from magpie.gripper import Gripper
import numpy as np

G = Gripper()

new_task = True

if new_task:
    G.reset_parameters()

goal_aperture = 5
complete_grasp = True
initial_force = (5 * 9.81) / 0.2
additional_closur

KeyboardInterrupt: Interrupted by user